In [ ]:
import pandas as pd

churn_data = pd.read_csv('churn.csv', sep=';')
print(churn_data.head())

import plotly
import plotly.express as px

# 9.1. Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

In [ ]:
churn_data['Exited_values'] = churn_data['Exited'].value_counts()
fig = px.pie(churn_data , values= 'Exited_values', title= 'Соотношение ушедших и лояльных клиентов')
fig.update_traces(textposition='inside' , textinfo='percent+label')
fig.show()

Соотношение лояльных клиентов к ушедшим 80/20. То есть уходит из банка каждый пятый клиент.

# 9.2. Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [ ]:
histogram_data = churn_data[churn_data['Balance'] > 2500]
fig = px.histogram(
    data_frame = histogram_data,
    x = 'Balance',
    height=300,
    width=700,
    title='Распределение баланса на счетах в банке (>2500$)',
)
fig.show()

Баланс пользователя начинается от 3999к до 251к. Больше всего клиентов в банке с балансом от 100к до 150к

# 9.3. Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [ ]:
fig = px.histogram(
    data_frame = churn_data,
    x = 'Balance',
    color='Exited',
    height=300,
    width=700,
    title='Распределение баланса на счетах в банке',
)
fig.show()

У большого числа ушедших из банка остался там баланс, аномалия? Так же суммы на счетах ушедших клиентов значительно меньше, чем на счетах активных клиентов. Значит банку нужно активнее привлекать денежные средства клиентов на свои счета.

# 9.4. Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [ ]:
fig = px.box(
    data_frame=churn_data,
    x='Age',
    color='Exited',
    height=300,
    width=700,
    title='Распределение возраста клиентов по группам'
)
fig.show()


Возраст ушедших клиентов колеблется от 18 до 71 года. Есть небольшое количество аномалий по возрасту: более 80 лет - но можно принять их за истину, так как возрастные клиенты по разным причинам покидают банки возраст лояльных клиентов колеблется от 18 до 56 лет. Очень большое количество аномальных выбросов от 56 до 92 лет нужна проверка данных

# 9.5. Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [ ]:
fig = px.bar(
    churn_data,
    x='CreditScore',
    y='EstimatedSalary',
    color='Exited',
    height=800
)

fig.show()

Зависимость проследить не удалось.

# 9.6. Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

In [ ]:
mask = churn_data['Exited'] == 1
GenderLost = churn_data[mask].groupby(['Gender'])['Exited'].count()

print(GenderLost)


Наибольший отток среди женщин

# 9.7. Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [ ]:
bar_data = churn_data.groupby(['NumOfProducts' , 'Exited'] , as_index=False).count()
fig = px.bar(
    data_frame=bar_data,
    x='NumOfProducts',
    y='RowNumber',
    color='Exited',
    barmode='group',
    height=300,
    width=200,
    title='Соотношение ушедших и лояльных клиентов в зависимости от числа услуг'
)
fig.show()

Всего банк предлагает своим клиентам 4 услуги. Доля ушедших клиентов больше всего среди тех, кто пользуется одним продуктом банка. На втором месте по доле ушедших клиентов - пользователи двух услуг. Доля лояльных пользователей, у которых подключено три услуги банка значительно ниже, чем доля ушедших. Среди пользователей 4мя услугами банка нет лояльных пользователей. Можно сделать вывод, что оптимальное количество услуг для клиентова банка = 2

# 9.8. Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [ ]:

churn_data['Active'] = churn_data.IsActiveMember.apply(lambda x: 'Активные клиенты' if x else 'Неактивные клиенты')
churn_data['Loyal'] = churn_data.Exited.apply(lambda x: 'Нелоялные клиенты' if x else 'Лояльные клиенты')

fig = px.sunburst(
    churn_data,
    path=['Active','Loyal'],
)
fig.update_traces(textinfo='label+percent entry')

fig.show()

Доля активных и неактивных клиентов примерно равны. Среди неактивных клиентов велика доля лояльных банку пользователей. Стоит обратить внимание на них, что то предложить, вовлечь в пользование счетом. Думаю, будет правильным программу лояльности направить в первую очередь на этих клиентов


# 9.9. В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

In [ ]:
chorn_data = churn_data.groupby(['Geography'], as_index=False).mean()
fig = px.choropleth(
    data_frame=chorn_data,
    locations='Geography',
    locationmode='country names',
    color='Exited',
    hover_name='Geography',
    color_continuous_scale='Reds',
    height=500,
    width=700,
    title='Отток клиентов по карте мира'
    )

fig.show()

Самый большой отток клиентов в Германии

# 9.10. Переведите числовой признак CreditScore в категориальный. Для этого воспользуйтесь функцией get_credit_score_cat(), которая приведена ниже. Примените её к столбцу CreditScore и создайте новый признак CreditScoreCat — категории кредитного рейтинга.

In [ ]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"
    
churn_data['CreditScoreCat'] = churn_data['CreditScore'].apply(get_credit_score_cat)

df_churn_data = churn_data.pivot_table(
    values='Exited',
    index='CreditScoreCat',
    columns='Tenure'
)

fig = px.imshow(df_churn_data)

fig.show()

Чаще всего уходят клиенты с плохим кредитным рейтингом, которые пользовались услугами банка менее года. Так же наблюдается довольно больой % ухода пользователей с плохим кредитным рейтингом и сроком пользования услугами банка от 10 ле